# **NLP FINAL PROJECT: Recipes Chatbot**

## Imports of the libraries needed

In [ ]:
USERINPUT = input()

TypeError: ignored

In [4]:
!pip install -U sentence-transformers
!pip install -q xlrd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.9 MB/s 
     |████████████████████████████████| 5.8 MB 55.8 MB/s 
     |████████████████████████████████| 1.3 MB 74.9 MB/s 
     |████████████████████████████████| 182 kB 88.6 MB/s 
     |████████████████████████████████| 7.6 MB 52.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=670af1a6757eacb814492af34036afbc5a2e30474358eb5f8aa899ffea2e2aeb
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [5]:
import pandas as pd
import unicodedata
import re
import random
import itertools
import nltk
import random

from sentence_transformers import SentenceTransformer, util
from itertools import chain
from nltk.stem import WordNetLemmatizer 

nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


## Datasets Loadings

In [6]:
# loadding the reciepes dataset
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/APUNTS/IntroduccióalesTécniquesdeProcessamentdeLlenguatgeNatural/Project
# %cd /content/drive/MyDrive/NLP

df_recipes = pd.read_csv('ReciepeDataset/RecipeNLG_dataset.csv')
with open('IngredientsDS/baseIngredients.txt') as f:
  lines = f.readlines()
  
ingredients_name_stopwords = lines[0].split(",")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/APUNTS/IntroduccióalesTécniquesdeProcessamentdeLlenguatgeNatural/Project


In [ ]:
# removing the unecessary columns of the dataset
df_recipes = df_recipes.drop(columns=['Unnamed: 0', 'source'])
# displaying the first 10 rows of the dataset
display(df_recipes.head(10))

,title,ingredients,directions,link,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."
5,Cheeseburger Potato Soup,"[""6 baking potatoes"", ""1 lb. of extra lean gro...","[""Wash potatoes; prick several times with a fo...",www.cookbooks.com/Recipe-Details.aspx?id=20115,"[""baking potatoes"", ""extra lean ground beef"", ..."
6,Rhubarb Coffee Cake,"[""1 1/2 c. sugar"", ""1/2 c. butter"", ""1 egg"", ""...","[""Cream sugar and butter."", ""Add egg and beat ...",www.cookbooks.com/Recipe-Details.aspx?id=210288,"[""sugar"", ""butter"", ""egg"", ""buttermilk"", ""flou..."
7,Scalloped Corn,"[""1 can cream-style corn"", ""1 can whole kernel...","[""Mix together both cans of corn, crackers, eg...",www.cookbooks.com/Recipe-Details.aspx?id=876969,"[""cream-style corn"", ""whole kernel corn"", ""cra..."
8,Nolan'S Pepper Steak,"[""1 1/2 lb. round steak (1-inch thick), cut in...","[""Roll steak strips in flour."", ""Brown in skil...",www.cookbooks.com/Recipe-Details.aspx?id=375254,"[""tomatoes"", ""water"", ""onions"", ""Worcestershir..."
9,Millionaire Pie,"[""1 large container Cool Whip"", ""1 large can c...","[""Empty Cool Whip into a bowl."", ""Drain juice ...",www.cookbooks.com/Recipe-Details.aspx?id=794547,"[""pineapple"", ""condensed milk"", ""lemons"", ""pec..."


In [ ]:
# # Dataframe with sentences and their Type

# dict = {'Question for recipe' : ['What can I cook with ___ and ___?',
#                                  'Do you know any recipe with ___?', 
#                                  'What can I do with ___ and ___?', 
#                                  'There is any recipe with ___?',
#                                  'I would like to do something with ___',
#                                  'I would enjoy something with ___',
#                                  'Do you know anything with ___ and ___?',
#                                  'I’m thinking of a recipe with ___',
#                                  'I have some ___ that I should use',
#                                  'I feel like cooking something with ___'
#                                  ],
#         'Question for suggestion' : ['What can I cook?',
#                                      'I want to try a new recipe',
#                                      'Give me a recipe',
#                                      'I am not inspired',
#                                      'Would u suggest me some recipe',
#                                      'I don’t know what to cock today',
#                                      'Could you suggest a recipe?',
#                                      'May you recommend me something to cock',
#                                      'Today i feel inspired to cock something good',
#                                      'I need ideas of recipes'
#                                      ]}

# df = pd.DataFrame(dict)
  
# # displaying the DataFrame
# # display(df)
# display(df)
# if 'recipe' in df.values:
#   print('exists')

In [ ]:
df_all = pd.read_excel('sentences_types_load_2.xlsx')
df_all
df_1 = df_all[[9,10]] # ingredient or suggestion
df_2 = df_all[[1,2,3,4,5]] # repeat last step , this step, don't want to continue , want other recipe or keep going
df_3 = df_all[[6,7]] # repeat step or link
df_4 = df_all[[12,13]] # new recipe or end conversation
df_5 = df_all[[6,13]] # ask for the link or end conversation
display(df_4)

,12.0,13.0
0,new recipe,No
1,I have more time today,I want to finish
2,I am inspired,"I am done, thanks"
3,I want to repeat,I want to leave
4,Another one,stop
5,Show me more dishes,I'm in a hurry
6,I am bored,That's it for today
7,I want to cook more,Nope
8,I feel like cooking,I had enough
9,I would like to prepare something else,Ciao


## Auxiliar Functions Definitions

In [ ]:
def getRandomRecipes():
  ids = []
  iterable = 0
  for iterable in range(5): ids.append(random.randint(0,len(df_recipes) - 1))
  recipes = []
  iterable = 0
  max_recipes = 5
  for id in ids:
    # select the entire row of the dataset
    recipes.append(df_recipes.loc[id])
    if max_recipes <= iterable : break
    else: iterable +=1
  return recipes

In [ ]:
# auxiliar function to clean the characters of the texts
def clean(text):
  # removing spaces
  text = text.strip()
  # converting to lowercase
  text = text.lower()
  # romanizing
  nfkd_form = unicodedata.normalize('NFKD', text)
  only_ascii  = nfkd_form.encode('ASCII', 'ignore')
  text = only_ascii .decode('utf-8') # bytes to str
  # removing anything that is not a letter or digit
  text = re.sub(r"[^a-zA-Z ]", "", text)
  # double spaces to single
  text = re.sub(' +', ' ', text)
  return text

In [ ]:
# auxiliar function to take the ingredients of each row 
def clean_ingr_list(ingredients_list):
  final_ingredient_list = []
  ingredients_list = ingredients_list.replace("[","")
  ingredients_list = ingredients_list.replace("]","")
  ingredients_list = ingredients_list.replace("\"","")
  ingredients_list = ingredients_list.split(",")
  for ingredient in ingredients_list: final_ingredient_list.append(ingredient.strip())
  return final_ingredient_list

In [ ]:
# Ingredients extraction
# Global variable where all the ingredients (keywords) in a list of list are saved to be able to search throgh them
ingredients = []

for ingredients_list in df_recipes['NER']:
  ingredients_list_cleared = clean_ingr_list(ingredients_list)
  ingredients.append(ingredients_list_cleared)

In [ ]:
# displaying the first 3 elements of the matrix of all ingredients of the dataset
display(ingredients[0:3])

[['brown sugar',
  'milk',
  'vanilla',
  'nuts',
  'butter',
  'bite size shredded rice biscuits'],
 ['beef', 'chicken breasts', 'cream of mushroom soup', 'sour cream'],
 ['frozen corn', 'cream cheese', 'butter', 'garlic powder', 'salt', 'pepper']]

In [ ]:
# Check if the input has any ingredient
# Auxiliar function to return the lists of ingredients (keywords) where the input is found
def ingredient_check(input):

    # the input is cleaned and splited in to words
    input = clean(input)
    input  = input.split(" ")

    ingredient_found_list = []

    for word in input:
      for ingredient_list in ingredients :
        #Try spacy
        if word in ingredient_list and (word in ingredients_name_stopwords or lemmatizer.lemmatize(word) in ingredients_name_stopwords):
          ingredient_found_list.append(word)
          break

    return ingredient_found_list

In [ ]:
# Testing the previous function
inop = "Okey, do you have something with eggs, beef and milk?"
ingredient_check_l = ingredient_check(inop)
print(ingredient_check_l)

['eggs', 'beef', 'milk']


In [ ]:
# # auxiliar function to intersect two lists
# def intersection(lst1, lst2):
#     temp = set(lst2)
#     lst3 = [value for value in lst1 if value in temp]
#     return lst3

In [ ]:
# # function to find in wich recipes appear the ingredients passed
# # list of ingredients passed as input
# def ingredient_finder(ingredients_input):

#   ing_ids_list = []
#   # For each ingredient we save in which idx it appears
#   for ingredient in ingredients_input:
#     ing_ids = []
#     idx = 0
#     #For each ingredients list of keywords
#     for ing_lst in ingredients:
#       #If the ingredient appears we save the index
#       if ingredient in ing_lst:
#         ing_ids.append(idx)
#       idx+=1
#     #At the end we append all the idx in a final list
#     ing_ids_list.append(ing_ids)

#   final_idx_list = []
#   #for each list of indexes we make an intersection
#   for n in range(len(ing_ids_list)):
#     if n == 0: final_idx_list = ing_ids_list[0]
#     else: final_idx_list = intersection(ing_ids_list[n],final_idx_list)
#   ing_idx = 0
#   #print(final_idx_list)
#   reciepes = []
#   max = len(final_idx_list) -1
#   if (max >= 0):
#     #print("POSSIBLE RECIEPES FOR THIS INGREDIENT: \n")
#     for row in df_recipes.values: 
#       if final_idx_list[ing_idx] == row[0]:
#         #print(row)
#         reciepes.append(row)
#         ing_idx+=1
#         #print("\n")
#         if ing_idx > max: break
#   return reciepes

In [ ]:
list1 = ['eggs','milk']
list2 = ['coock','balls','eggs','chocolate']
list3 = ['pasto','eggs','coock','balls','chocolate']
list4 = ['milk', ' eggs', ' Bisquick', ' sugar', ' salt', ' margarine', ' vanilla', ' coconut']
if(set(list1).issubset(set(list2))): print("list 1 is a subset of list 2")
if(set(list1).issubset(set(list3))): print("list 1 is a subset of list 3")
if(set(list1).issubset(set(list4))): print("list 1 is a subset of list 4")

In [ ]:
# Auxiliar function to obtain a list of ids where the ingredients
# passed as input appear
def recipes_ids(ingredients_input):
  recipes_ids = []
  id = 0
  for key_ingredients in ingredients:
    # if the input is in the selected list
    if(set(ingredients_input).issubset(set(key_ingredients))):
      recipes_ids.append(id)
    id += 1
    
  return recipes_ids 

In [ ]:
# Auxiliar function to get all the information of the recepies where
# the ingredients list input appear
def recipes(ingredients_input):
  # getting the ids with the function recipes_ids()
  ids = recipes_ids(ingredients_input)
  recipes = []
  iterable = 0
  max_recipes = 5
  #SUFFLE
  for id in ids:
    # select the entire row of the dataset
    recipes.append(df_recipes.loc[id])
    if max_recipes <= iterable : break
    else: iterable +=1
  return recipes

In [ ]:
# auxiliar function to get the link of the recipe
def get_link(recipe_id):
  return df_recipes.loc[recipe_id]['link']

In [ ]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# Auxiliar function to get the type of the question (using sentence similarity)
def sen_type(df_sentences, input):
  # computing the embedding of the input
  input_embed = model.encode(input, convert_to_tensor=True)

  # Creation of a dataframe with sentence similarity between the input 
  # and the sentences in df_sentences dataframe
  column_names = ["Sentence", "Type", "Score"]
  similarities = pd.DataFrame(columns = column_names)
  
  idx = 0


  for (colname,colval) in df_sentences.iteritems():
    for i in range(len(colval.values)):
        # compute embedding of the sentence in the df
        sen_embed = model.encode(colval.values[i], convert_to_tensor=True)
        # computation of the sentence similarity
        score = float(util.pytorch_cos_sim(input_embed, sen_embed))
        # store the values in similarities dataframe
        similarities.loc[idx] = [colval.values[i], colname, score]
        idx += 1

  # sorting the dataframe by the value 'Score' in ascending order
  similarities = similarities.sort_values(by='Score', ascending=False)
  #print(similarities)
  #print(similarities.drop_duplicates(subset='Type'))
  # return the type of the first row
  return similarities.iloc[0][1]

In [ ]:
# testing the previous function
input = "I want to coock something with __ "
print(sen_type(df, input))

question for recipe (given ingredients)


## Chatbot Implementation

In [ ]:
#MAIN COMPONENT
# variable que guarda els turns
turn = 0
mode = 0
#var to be able to keep a track of the recipe step
recipe_step_counter = 0
first = 0
#var to to be able to show other reciepes to the user and keep track of the current recipe
recipe_idx_count = 0
user_inputs = []
#list of possible recipes given the user ingredients
possible_recipes = []

userInput = "I looking for some delicious meal to cook today"
while(True):
  if mode == 0: # inicialitzar conversació i classificar el tipus de input
    # start = False
    # inicialitzem l'index de la llista de receptes
    recipe_idx_count = 0
    # fer que aquesta frase només es digui a l'inici
    if turn == 0:
      print("Hello! I am your culinary assistant. What do you want to cook? Any preferences or ingredients?\n")
      turn = 1
    # InputQuestion = input('Write your response:\n')
    print(userInput,"\n")
    # first of all, we check if there are any specific ingredients on the input
    # and store it
    ing = ingredient_check(userInput)
    print(ing)
    # if there are ingredients in the input
    if (len(ing) > 0):

      # search for recipes with those specific ingredients
      possible_recipes = recipes(ing)
      if (len(possible_recipes) > 0):

        recipe_title = possible_recipes[recipe_idx_count][0]
        print("Are you interested in the recepie %s? (Answer with Yes or No)\n" %recipe_title)
        userInput = "Yes"
        print("Yes\n")
        mode = 1
       # break

      else:
        print("I haven't found any reciepe with this ingredients. Please, change ingredient names or if you need a suggestion enter 1.\n")
        # userInput = input()
        break
        if userInput == '1':
          mode = 4
        else:
          mode = 0
    else:
      mode = 4
      
      # si l'usuari no dona ingredients, passar a un mode
      # entenc que s'ha de fer sentence similarity

  #APPLY DIALOGUE TO KNOW IF ITS YES OR NOT
  #SWITCHBOARD
  
  # MODE 1 --> resposta si/no
  if mode == 1:
    #yes_no_input = "yes" # input()
    yes_no_input = clean(userInput)
    if yes_no_input == "yes":
      print("Yes")
      mode = 2
    elif yes_no_input == "no":
      print("No")
      mode = 6
    else:
      #We apply sentece similarity to know what the user wants
      break

  # MODE 2 --> donar ingredients recepte
  if mode == 2:
    ingrednients_show = possible_recipes[recipe_idx_count][4]
    print("Okey, these are the recipe ingredients %s, shall we continue?(Answer with Yes or No)\n" %(ingrednients_show))
    userInput = "Yes"
    print(userInput)
    if userInput == "Yes":
      recipe_step_counter = 0
      mode = 3
      
    else:
      print("Fine, do you want me to show you more recipes or do you want to try with new ingredients?")
      #pordiem utilitzar sentence similarity aqui també
      #if l'usuari vol mes receptes i no les ha vist totes ja mode = 6
      #if l'usuari vol canviar d'idea sobre els ingredients mode = 0
      break
  # MODE 3 --> donar recepte per pasos
  if mode == 3:
    # if the orders just started
    if (recipe_step_counter == 0 and first == 0):
      first = 1
      print("Perfect, let's start \n If you get lost or you need help, please let me now \n")
      #print("Step %s : %s\n"(recipe_step_counter,possible_recipes[recipe_idx_count][1]))
      # print("\n",possible_recipes[recipe_idx_count][2][0])
      # retaining and cleaning the directions
      directions = possible_recipes[recipe_idx_count][2]
      directions = directions.replace("]", "")
      directions = directions.replace("[", "")
      directions = directions.replace(",", "")
      directions = directions.replace('"', "")
      directions = directions.split(".")
      # we reomve the last element that is a ' '
      directions = directions[0:len(directions)-1]
    # if the orders haven't finished

    if(recipe_step_counter < len(directions)-1):
      print("Step %s : %s\n" %(recipe_step_counter, directions[recipe_step_counter]))
      recipe_step_counter += 1
      print("\n If you want to continue enter 1.")
      userInput = "going good rn"
      print(userInput,"\n")
      if userInput == "1":
        mode == 3
      else:
        mode = 9

    # the user has finished the recipe
    if (recipe_step_counter == len(directions)-1):
      mode = 8

    # break

    # Tenir en compte si no ha entes algo

  # MODE 4 --> suggestion 
    # demanar ingredients o recepte random
  if mode == 4:
    print("Fine, I can recommend you some nice recipes\n")
    possible_recipes = getRandomRecipes()
    recipe_title = possible_recipes[recipe_idx_count][0]
    print("Are you interested in the recepie %s? (Answer with Yes or No)\n" %recipe_title)
    userInput = "No"

    mode = 1
  # MODE 5 --> link
  if mode == 5:
    print("Here you have the original source of the recipe %s: %s \n" %(recipe_title, get_link(recipe_idx_count)))
    turn += 1
    break
    # if the conversation hasn't lasted long enough
    if turn < 10:
      mode = 7
    else:
      mode = 8
  # MODE 6 --> canviar de recpete (no li agrada)
  if mode == 6:
    recipe_idx_count +=1
    if len(possible_recipes) > recipe_idx_count:
      recipe_title = possible_recipes[recipe_idx_count][0]
      print("Are you interested in the recipe %s? (Answer with Yes or No)\n" %recipe_title)
      mode = 1
    else:
      print("I don't have any more recipes with that characteristics (ingredients), could you choose any other ingredients?\n")
      mode = 0
      userInput = "Okey, do you have something with eggs?\n"
    
  # MODE 7 --> allargar la conversació ?¿?¿?¿
  # MODE 8 --> ha acabat, pregunta si vol fer alguna cosa més o seguir cuinant
  if mode == 8:
    print("Perfect, do you want to start another recipe, just finish?")
    userInput = "No, i'm done"
    print(userInput,"\n")
    result = int(sen_type(df_4, userInput))
    print(result)
    #if dels resutls
    if result == 12: 
      mode = 0
      print("Perfect, do you want any suggestion or do you want a specific dish (ingredient/s)?\n")
    if result == 13: mode = 11


  # MODE 9 --> algo ha pasat mostrant la recepta
  if mode == 9:
    #apply sentence similarity
    result = int(sen_type(df_2, userInput))
    print(result)
    if result == 1: print("please let me know in which step did you got lost\n") 
    if result == 2: mode = 10
    if result == 3: print("Okey, Show other recipe from the list....\n")
    if result == 4: print("Okey, so do you want to end the conversation?\n")
    if result == 5: mode = 3

  # else: 
  if mode == 10:
    print("do you want me to repeat it? Or I can provide you with a link if its more clear for you\n")
    userInput = "repeat pls"
    result = int(sen_type(df_3, userInput))
    print(userInput,"\n")

    if result == 7:
      recipe_step_counter -= 1
      mode = 3
    else:
      mode = 5
  if mode == 11:
    print("Before finishing, do you want to keep the recipe?\n")
    userInput = "Yes please"
    print(userInput,"\n")
    result = int(sen_type(df_5, userInput))
    if result == 6:
      print("Here you have the original source of the recipe %s: %s \n" %(recipe_title, get_link(recipe_idx_count)))
    break
    

Hello! I am your culinary assistant. What do you want to cook? Any preferences or ingredients?

I looking for some delicious meal to cook today 

[]
Fine, I can recommend you some nice recipes

Are you interested in the recepie Lemon Pound Cake? (Answer with Yes or No)

No
Are you interested in the recipe Obert'S Vegetable Casserole? (Answer with Yes or No)

No
Are you interested in the recipe Mini Quiches? (Answer with Yes or No)

No
Are you interested in the recipe Orange Glazed Halibut? (Answer with Yes or No)

No
Are you interested in the recipe Kahlua Cake? (Answer with Yes or No)

No
I don't have any more recipes with that characteristics (ingredients), could you choose any other ingredients?

Okey, do you have something with eggs?
 

['eggs']
Are you interested in the recepie Grandma Hanrath'S Banana Breadfort Collins, Colorado  ? (Answer with Yes or No)

Yes

Yes
Okey, these are the recipe ingredients ["sugar", "shortening", "eggs", "salt", "soda", "flour", "nuts", "bananas"], 

## Tests

In [ ]:
userInput = "I'm looking to cock a beef that i have in my refrigerator"
ing = ingredient_check(userInput)
print(ing)
possible_recipes = recipes(ing)
recipe_title = possible_recipes[0][0]
print("Recipe title:", recipe_title)
ingrednients_show = possible_recipes[0][4]
print(ingrednients_show)
print(possible_recipes[0][2])

['beef']
Recipe title: Jewell Ball'S Chicken
["beef", "chicken breasts", "cream of mushroom soup", "sour cream"]
["Place chipped beef on bottom of baking dish.", "Place chicken on top of beef.", "Mix soup and cream together; pour over chicken. Bake, uncovered, at 275\u00b0 for 3 hours."]


In [ ]:
directions = possible_recipes[0][2]
print(directions)
directions = directions.replace("]", "")
directions = directions.replace("[", "")
directions = directions.replace(",", "")
directions = directions.replace('"', "")
directions = directions.split(".")
# we reomve the last element that is a ' '
directions = directions[0:len(directions)-1]

print(directions)

["Place chipped beef on bottom of baking dish.", "Place chicken on top of beef.", "Mix soup and cream together; pour over chicken. Bake, uncovered, at 275\u00b0 for 3 hours."]
['Place chipped beef on bottom of baking dish', ' Place chicken on top of beef', ' Mix soup and cream together; pour over chicken', ' Bake uncovered at 275\\u00b0 for 3 hours']


In [ ]:
# inputs = ["I would like to eat something with eggs","Can you recomend me any reciepe?","Can you recomend me some reciepe with butter, milk and chocolate?","In my refrigerator I have some tomatoes and cucumber, could you recommend me something whith that?"] 


In [ ]:
posible_reciepes = recipes(["milk","vanilla","nuts","flour","oatmeal","egg","cinnamon","soda"])
# posible_reciepes = ingredient_finder(['oven'])
# recipes_with_ingredients
k = 0
max = 10
if len(posible_reciepes) < max: max = len(posible_reciepes)
for k in range(max):
  print(posible_reciepes[k],"\n")

#IMPORTANTE, ALGO FUNCIONA MAL CUAND HAY MUCHOS INGREDIENTES, NO VEO EL PORQUÉ

title                                            Oatmeal Hermits
ingredients    ["3 c. oatmeal", "1 c. flour", "1 c. margarine...
directions     ["Mix all together well.", "Bake at 375\u00b0 ...
link             www.cookbooks.com/Recipe-Details.aspx?id=370786
NER            ["oatmeal", "flour", "margarine", "brown sugar...
Name: 12576, dtype: object 

title                                               M&M's Pizzas
ingredients    ["3/4 cup butter", "1 cup brown sugar", "1/2 c...
directions     ["Cream butter and sugar.", "Add eggs, milk an...
link                              www.foodgeeks.com/recipes/2514
NER            ["butter", "brown sugar", "sugar", "egg", "mil...
Name: 2006529, dtype: object 

